In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

思路：可以增加的features
1. 是否同一ip地址多个帐号
2. 登录的平均间隔时间、最短间隔时间、最大间隔时间
3. 一天中的交易次数

In [15]:
trade_df = pd.read_csv('./data/Risk_Detection_Qualification/t_trade.csv', index_col='rowkey', dtype={'id': np.str})
login_df = pd.read_csv('./data/Risk_Detection_Qualification/t_login.csv', index_col='log_id', dtype={'id': np.str, 'timestamp': np.str})
trade_test_df = pd.read_csv('./data/Risk_Detection_Qualification/t_trade_test.csv', index_col='rowkey', dtype={'id': np.str})
login_test_df = pd.read_csv('./data/Risk_Detection_Qualification/t_login_test.csv', index_col='log_id', dtype={'id': np.str, 'timestamp': np.str})

In [16]:
# 得到一列不同的值
# login_df['result'].unique()
# login_df['id'].unique()
# len(login_df['city'].unique()), len(login_df.index)
# login_df['is_risk'].count()
trade_df.shape, trade_df[trade_df['is_risk']==0].shape, trade_df[trade_df['is_risk']==1].shape

((132719, 3), (129076, 3), (3643, 3))

In [17]:
trade_df_by_id = trade_df['is_risk'].groupby(trade_df['id'])

id_risk_trade   = trade_df[trade_df['is_risk']==1]['id']
id_nonrisk_trade = trade_df[trade_df['is_risk']==0]['id']
checked_id = set(id_risk) & set(id_nonrisk)
len(id_risk), len(id_nonrisk), len(checked_id)

(3643, 129076, 455)

In [20]:
login_df_by_id = login_df['ip'].groupby(login_df['id']).count()
print(login_df_by_id)
# '175732' in login_df['id']
# login_df['id']

id
100002      9
100006     21
100019      4
100020     36
100024     11
100027      3
100039     25
100046     45
100058     18
100063     10
100068      9
100070      9
100077     23
100078      3
100087     51
10009       2
100093     81
100101      5
100108      5
100114     13
100115      9
100127      7
100138      3
100142     18
100154    103
100158      6
100169     89
100172      2
100176     17
100180      3
         ... 
99823       2
99825      10
99828      56
9983        6
99832       8
9984       25
99840      78
99848       1
99853      12
99855      50
99856      22
99861       2
99871      11
99883       1
99899       8
9990        1
99901       7
99906       3
9991       15
99937       5
99938      17
99943       3
99944     125
99950      53
9996       16
99962       4
99976     117
99977      30
9998        6
99987       9
Name: ip, dtype: int64


In [21]:
login_df['timestamp'].unique()

array([  1.42864166e+09,   1.42880884e+09,   1.42770470e+09, ...,
         1.42967938e+09,   1.42753038e+09,   1.42344511e+09])